In [ ]:
## Market and Asset
exchange_name = 'binance'
asset_symbol = 'BTCUSDT'

from modules.assets_handler import AssetsHandler
from modules.klines_handler import OneMinuteCandlesAmount, Transform
from pydoc import locate
import time

##Database credentials
DB_HOST = 'localhost'
DB_PORT = 5432
DB_NAME = 'pandazord_database'                      
DB_USER = 'pandazord'
DB_PASSWORD = 'QBBV9E%pcYKHUcjj'

#Como este instanciamento é apenas "leitura", pode ser comum ao "trader real" e ao backing test. As diferenças:
    ## A) Trader real tem sleep, backing test não
    ## B) Trader real diapara ordem para corretora, backing test não
    ## C) Trader real atualiza entradas no banco, backing test não (a menos dos dados de interesse para o 
        # histótico)

#Essa instância deve ser "recriada" para o backing test, ou pelo.
Asset = AssetsHandler(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME, exchange_name, asset_symbol)

#Operational Parameters
op_param = Asset.info().iloc[0].operational_parameters
strategy_param = op_param['strategy']['parameters']
stop_param = op_param['stop_loss']['parameters']

strategy_name = op_param['strategy']['name']
stop_name = op_param['stop_loss']['name']

Strategy = locate('modules.strategies.' + strategy_name)(strategy_param)
StopLoss = locate('modules.stop_loss.' + stop_name)(stop_param)
OrderHandler = locate('modules.order_handler' + exchange_name.capitalize() + 'Handler')
KlinesHandler = locate('modules.klines_handler.' + exchange_name.capitalize() 
                       + 'FromDb')(DB_USER, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME)

In [ ]:
# NEW: Draft of asset_hnadler out

## Operational parameters
op_param = Asset.operational_parameters ### Pode continuar sendo um dicionário com parâmetros, a fim de evitar a 
                                        #   criação de muitos objetos
## Position
### É um objeto que, difrentemente do 'operational parameters' que só será consultado, será modificado pelo 
###'trader.' Então é prudente criar-se uma subclasse para este objeto


In [ ]:
# NEW: Trader as class

class Trader:
    
    def __init__(self, exchange_name, asset_symbol):
        pass
    
    def for_real(self):
        pass
    
    def backing_test(self):
        pass
        
    

In [ ]:
Asset.info()

In [ ]:
# Teste estratégia

n_candles_to_strategy_analisys = Strategy.how_many_candles()

klines_to_strategy_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                       candle_interval=strategy_param['candle_interval'], 
                                                       number_of_candles=n_candles_to_strategy_analisys)

trade = Strategy.verify(klines_to_strategy_analisys, Asset.info().position.iloc[0])

print('Últimas klines')
print(' ')
print(klines_to_strategy_analisys.tail())
print(' ')
print('Posição atual.....................: ', Asset.info().position.iloc[0]['side'])
print('Preço alvo Atual..................: ', Asset.info().position.iloc[0]['target_price'])
print('Posição e alavancagem definidos....: ', Strategy.what_side_and_leverage(klines_to_strategy_analisys))
print('Trade.............................: ', trade.is_true)
print('Ordem dada........................: ', trade.command)
print('Só para teste, qual a alavancagem.: ', trade.leverage)

In [ ]:
#Teste stop

n_candles_to_stop_analisys = StopLoss.how_many_candles()

klines_to_stop_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                   candle_interval=stop_param['candle_interval'], 
                                                   number_of_candles=n_candles_to_stop_analisys)

print('Últimas klines')
print(' ')
print(klines_to_stop_analisys.tail())

#A fim de testar o stop, tenho de fazer o stop aparecer... portanto é preciso, 
#não só arbitrar o preço de stop, como "rodar" entre klines passadas. Talvez 
#seja até mais útil contruir o 'backing test' inteiro

stop = StopLoss.verify(klines_to_stop_analisys, Asset.info().position.iloc[0])

In [ ]:
# Ação do trader por ciclo (real)

#while true
n_candles_to_stop_analisys = StopLoss.how_many_candles()
n_candles_to_strategy_analisys = Strategy.how_many_candles()
sleep_time = op_param['sleep_duration'] 
is_positioned = bool(Asset. #TODO: Rever, pois pode evocar direto o 'position', ao
                                                         #   invés do 'Asset.position'...

if (is_positioned):
    
    sleep_time = (60 - int(datetime.now().second))
    klines_to_stop_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                       candle_interval=stop_param['candle_interval'], 
                                                       number_of_candles=n_candles_to_stop_analisys)
    
    stop = StopLoss.verify(klines_to_stop_analisys, Asset.info().position)
    
    if (stop.is_true):
        
        order = OrderHandler.execute(stop)
        time.sleep(10)
        order_confirmation = OrderHandler.check(order)
        Asset.update_position(order_confirmation)
    
    if(stop.update_target): Asset.update_position(stop.new_target_price)

klines_to_strategy_analisys = KlinesHandler.get_latest(asset_symbol, 
                                                       candle_interval=strategy_param['candle_interval'], 
                                                       number_of_candles=n_candles_to_strategy_analisys)

trade = Strategy.verify(klines_to_strategy_analisys, Asset.info().position)

if (trade.is_true):

    order = OrderHandler.execute(trade)
    time.sleep(10)
    order_confirmation = OrderHandler.check(order)
    Asset.update_position(order_confirmation)

time.sleep(sleep_time - int(datetime.now().second))

In [ ]:
# Ação do trader por ciclo (backing test)

## 0) Pré-ciclo: 
    
    ### 0.1) Instanciar todas as klines de 1m disponíveis copiar o asset.info()
all_1m_klines = KlinesHandler.all_latest_one_minute(asset_symbol)

    ### 0.2) Copiar o asset.info()
backing_test_asset = Asset.info().copy()

    ### 0.3) "Zerar" a posição
    #### OBS.: É melhor criar um objeto aqui (portar para o 'assets_handler'?) para manutenção da mesma estrutura
    #          do trader real
backing_test_asset.loc[0, 'position']['side'] = 'new_close'
backing_test_asset.loc[0, 'position']['size'] = 0.01
backing_test_asset.loc[0, 'position']['target_price'] = 0.04

position = BackingTestAssetPosition()

    ### Checkar stop
    ### ...
    ### fake_all_1h_klies = Transform(all_1m_klines[from_index:to_index]).from_1m_to('1h')

In [ ]:
class BackingTestAssetPosition:
    
    def __init__(self):
        self._side = 'close'
        self._size = 0.0
        self._target_price = 0.0
    
    @property
    def side(self):
        return self._side
    
    @property
    def size(self):
        return self._size
    
    @property
    def target_price(self):
        return self._target_price
    
    @side.setter
    def side(self, side_to_set):
        side_list = ['closed', 'long', 'short']
        error_message = 'Incorrect! Side must be '
        error_append = ''
        
        if (side_to_set not in side_list):
            for item in side_list[:-1]:
                error_append+= "'" + item + "'" + ', '
            
            error_message = error_message + error_append + 'or ' + "'" + side_list[-1] + "'" + '.'
    
            print(error_message) #TODO: Tratar exceção sem print
        else:
            self._side = side_to_set


    @size.setter
    def size(self, size_to_set):
        numerical = (isinstance(size_to_set, float) or isinstance(size_to_set, int))
        
        if (numerical):
            if (size_to_set < 0.0):
                print ('Invalid. Size must not be negative.')
        
            else:
                self._size = size_to_set
        
        else:
            print("Invalid. Must be a numerical ('float' or 'int') type." )

In [ ]:
TestAsset = BackingTestAssetPosition()

In [ ]:
TestAsset.size = 'jesus'

In [ ]:
TestAsset.size

In [ ]:
## Operational
operational_parameters = {
    'strategy' : {
        'name' : 'CrossSMA',
        
        'parameters' : {
            'candle_interval': '1h',
            'price_source' : 'ohlc4',
            'n_smaller' : 3,
            'n_bigger' : 100
        }
    },
    
    'stop_loss' : {
        'name' : 'Default',
        
        'parameters': {
            'candle_interval': '1m',
            'price_source' : 'ohlc4',
            
            'first_trigger' : {
                'rate(%)' : 5,
                
                'treshold' : {
                    'n_measurements' : 10,
                    'n_positives' : 3
                }
                
            },
            
            'second_trigger' : {
                'rate(%)' : 1,
                
                'treshold' : {
                    'n_measurements' : 50,
                    'n_positives' : 20
                }                
            },
            
            'update_target_if' : {
                'rate(%)' : 6,
                
                'treshold' : {
                    'n_measurements' : 10,
                    'n_positives' : 4                
                }
            }
        }
    },
    
    'sleep_duration': 60,
}

def default_operational_parameters():
    
    return operational_parameters

# Trader

## Real

1 - Pergunta à estratégia quantas amostras, de qual 'candle_interval', no mínimo, são necessárias para o cálculo da posição

2 - Solicita as klines correspondentes

3 - Envia as klines e o 'Asset.info()' ao módulo de estratégia para análise, recolhendo o sinal correspondente

4 - Para sinal diferente de 'hold', aciona o 'order_handler'

5 - Dorme

6 - 'Refresca' as klines

7 - Repete a partir do passo 3

## Backing Test

1 - Pede todas as klines

2 - Pergunta à estratégia quantas amostras, de qual 'candle_interval', no mínimo, são necessárias para o cálculo da posição

3 - Envia a fatia (suficientemente longa) de klines mais antiga e o 'Asset.info()' ao módulo de estratégia para análise, recolhendo o sinal correspondente

4 - Para sinal diferente de 'hold' procede a 'ordem fictícia'

5 - 'Avança uma' kline, 'refrescando' a fatia

6 - Repete a partir do passo 3

## Observações

1 - É a estratégia que instancia o stop  
2 - Existe um problema, ainda sem causa descoberta: o módulo 'klines_handler' está retornando uma kline a menos  
    
    2.1 - Solução sugerida:  
    
    Em OneMinuteCandlesAmount.total(self, out_candle_number), 'pedir um out_candle_number a mais, truncando em 
    'get_latest' para o número de candles desejado